In [11]:
import pandas as pd
import json

In [12]:
# Load data
players = pd.read_csv("data/raw_players.csv", parse_dates=["first_seen_at"])
events = pd.read_csv("data/raw_game_events.csv", parse_dates=["event_time"])
sessions = pd.read_csv("data/raw_sessions.csv", parse_dates=["session_start", "session_end"])

In [13]:
# Find the most recent user (by first_seen_at)
most_recent_player = players.loc[players["first_seen_at"].idxmax()]
most_recent_player_id = most_recent_player["player_id"]

print(f"Most recent player: {most_recent_player_id}")
print(f"First seen at: {most_recent_player['first_seen_at']}")
print(f"Country: {most_recent_player['country']}, Difficulty: {most_recent_player['difficulty_selected']}")
print(f"Number of sessions: {sessions[sessions['player_id'] == most_recent_player_id].shape[0]}")

Most recent player: player_21
First seen at: 2026-01-28 08:40:46.532138
Country: FR, Difficulty: normal
Number of sessions: 1


In [14]:
most_recent_player.head()

player_id                               player_21
first_seen_at          2026-01-28 08:40:46.532138
country                                        FR
language                                       fr
difficulty_selected                        normal
Name: 20, dtype: object

In [15]:
print("\n" + "="*60)
print("Event Flow:")
print("="*60)

# Get all events for this player, sorted by time
player_events = events[events["player_id"] == most_recent_player_id].sort_values("event_time")

# Parse properties JSON for better display
player_events_display = player_events.copy()
player_events_display["properties"] = player_events_display["properties"].apply(
    lambda x: json.loads(x) if isinstance(x, str) else x
)

# Display event flow
for idx, event in player_events_display.iterrows():
    props_str = ", ".join([f"{k}={v}" for k, v in event["properties"].items()]) if event["properties"] else "{}"
    print(f"{event['event_time']} | {event['event_name']:20s} | {props_str}")

print(f"\nTotal events: {len(player_events)}")


Event Flow:
2026-01-28 14:40:46.532138 | game_started         | load_time_ms=4064, resolution=4K, fps_target=30, audio_quality=high
2026-01-28 14:47:37.532138 | chapter_started      | chapter_id=1, chapter_name=The Outskirts, location=school, weather=fog, time_of_day=dawn
2026-01-28 14:47:47.532138 | enemy_killed         | chapter_id=1, enemy_type=infected, enemy_name=runner, weapon_type=rifle, weapon_name=hunting_rifle, damage_dealt=200, headshot=False, distance_meters=6, xp_gained=27, stealth_kill=False
2026-01-28 14:54:38.532138 | item_crafted         | chapter_id=1, item_type=molotov, materials_used=['alcohol', 'rag', 'bottle'], crafting_time_seconds=7, success=True, workbench_used=False
2026-01-28 14:56:51.532138 | chapter_started      | chapter_id=5, chapter_name=The University, location=park, weather=snow, time_of_day=night
2026-01-28 14:57:32.532138 | enemy_killed         | chapter_id=5, enemy_type=infected, enemy_name=stalker, weapon_type=bow, weapon_name=hunting_bow, damage_